### Load all necessary libraries

In [25]:
from nltk.corpus import stopwords
import csv,re
import pandas as pd
import string
import nltk
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy

In [26]:
# import spacy.cli
# spacy.cli.download("en_core_web_sm")
nlp = spacy.load('en_core_web_sm')

# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('punkt')

### To ignore character dring tokenizer

In [27]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

### If csv pre processing needed

In [28]:
# with open('stsbenchmark/sts-train.csv', 'r', encoding = 'utf-8') as infile, open('repaired.csv','w', encoding='utf-8') as outfile:
#     for line in infile.readlines():
#         error = 0
#         try:
#             line = line.replace('","', '')
#             # line = line.replace('",', '')
#             outfile.write(line)
#         except:
#             error += 1
        
#     print(error)

### Now csv to dataframe and data Pre-processing

In [29]:
with open('repaired.csv', 'r', encoding = 'utf-8-sig') as file:
    text1 = []
    text2 = []
    # check = []
    reader = csv.reader(file)
    count=1
    error = 0
    for row in reader:
        first_element = row[0]
        # check.append(first_element)
        
        try:
            sent2 = first_element.split('\t')[6]
            sent1 = first_element.split('\t')[5]
            
            sent1 = re.sub(r'\[[0-9]*\]',' ',sent1)
            sent1 = re.sub(r'\s+',' ',sent1)
            sent1 = sent1.lower()
            sent1 = tokenizer.tokenize(sent1)
            
            for i in range(len(sent1)):
                sent1 = [word for word in sent1 if word not in stopwords.words('english')]
            
            sent2 = re.sub(r'\[[0-9]*\]',' ',sent2)
            sent2 = re.sub(r'\s+',' ',sent2)
            sent2 = sent2.lower()
            sent2 = tokenizer.tokenize(sent2)
            
            for i in range(len(sent2)):
                sent2 = [word for word in sent2 if word not in stopwords.words('english')]
            
            text2.append(sent2)
            text1.append(sent1)
        except:
            error+=1 
        
    print(error) ### error represent, how much data it skips

210


In [30]:
import pandas as pd

data = pd.DataFrame({'text1': text1, 'text2': text2})

copydata=data.copy()
copydata.shape

(5005, 2)

In [31]:
copydata.head()

,text1,text2
0,"[plane, taking]","[air, plane, taking]"
1,"[man, playing, large, flute]","[man, playing, flute]"
2,"[man, spreading, shreded, cheese, pizza]","[man, spreading, shredded, cheese, uncooked, p..."
3,"[three, men, playing, chess]","[two, men, playing, chess]"
4,"[man, playing, cello]","[man, seated, playing, cello]"


### Count vectorizer similarity and Tf IDF similarity

In [14]:
def count_vcr():
  for i in range(len(copydata)):
    doc1=copydata['text1'][i]
    doc2=copydata['text2'][i]
    docs=(doc1,doc2)
    matrix = CountVectorizer().fit_transform(docs)
    cosine_sim = cosine_similarity(matrix[0], matrix[1])
    similarity.append(cosine_sim)
  return similarity

def similarity_fn():
  for i in range(len(copydata)):
    doc1=copydata['text1'][i]
    doc2=copydata['text2'][i]
    docs=(doc1,doc2)
    tfidf_matrix = tfidf_vectorizer.fit_transform(docs)
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
    similarity.append(cosine_sim)
  return similarity

### Count Vecotrizer and Cosine Similarity

In [32]:
bow_converter = CountVectorizer()
copydata.pipe(detoken).pipe(remove_space)
similarity=[]
similarity=count_vcr()
data_cvr=copydata.copy()
data_cvr['Similarity']=similarity
data_cvr[:5]

,text1,text2,Similarity
0,plane taking,air plane taking,[[0.816496580927726]]
1,man playing large flute,man playing flute,[[0.8660254037844388]]
2,man spreading shreded cheese pizza,man spreading shredded cheese uncooked pizza,[[0.7302967433402215]]
3,three men playing chess,two men playing chess,[[0.75]]
4,man playing cello,man seated playing cello,[[0.8660254037844388]]


### Tf-Idf Vecotrizer and Cosine Similarity

In [34]:
tfidf_vectorizer = TfidfVectorizer()
similarity=[]
similarity=similarity_fn()
data_tf=copydata.copy()
data_tf['Similarity']=similarity

In [36]:
all_data=data_cvr.copy()
all_data['Count-Vec Similarity']=all_data['Similarity']
all_data=all_data.drop('Similarity',axis=1)
all_data['Tf-idf Similarity']=data_tf['Similarity']
all_data

,text1,text2,Count-Vec Similarity,Tf-idf Similarity
0,plane taking,air plane taking,[[0.816496580927726]],[[0.7092972666062739]]
1,man playing large flute,man playing flute,[[0.8660254037844388]],[[0.7765145304745156]]
2,man spreading shreded cheese pizza,man spreading shredded cheese uncooked pizza,[[0.7302967433402215]],[[0.5803329846765686]]
3,three men playing chess,two men playing chess,[[0.75]],[[0.6029748160380572]]
4,man playing cello,man seated playing cello,[[0.8660254037844388]],[[0.7765145304745156]]
...,...,...,...,...
5000,severe gales storm clodagh hits britain,merkel pledges nato solidarity latvia,[[0.0]],[[0.0]]
5001,dozens egyptians hostages taken libyan terrori...,egyptian boat crash death toll rises bodies fo...,[[0.0]],[[0.0]]
5002,president heading bahrain,president xi china continue help fight ebola,[[0.2182178902359924]],[[0.12536693798731732]]
5003,china india vow bilateral ties,china scrambles reassure jittery stock traders,[[0.18257418583505539]],[[0.10163066979112656]]
